In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import random
import numpy as np

random.seed(7)

In [2]:
# from gmu_scraping import getFiles

# getFiles().get_audio()

In [16]:
from speech_modeling import h_model, h_input
raw_df = pd.read_csv("./formants.csv")
input_data = h_input(raw_df)
input_data.input_df[input_data.f1v_features]

                                                           value              \
variable                                                      F1               
word                                                           a               
phoneme                                                      AH0         EY1   
id         gender age location                                                 
english1   m      42  pittsburgh_pennsylvania_usa     484.116707    0.000000   
english10  f      35  davenport_iowa_usa              446.479293    0.000000   
english101 f      18  toronto_ontario_canada          500.414386    0.000000   
english102 m      22  torrington_connecticut_usa      454.650048    0.000000   
english103 m      21  statenisland_newyork_usa        440.677259    0.000000   
...                                                          ...         ...   
english95  m      18  mishawaka_indiana_usa           373.439207  436.228222   
english96  m      31  pointpleasant_newj

,F1-a-AH0,F1-a-EY1,F1-also-AO1,F1-also-OW0,F1-and-AE1,F1-and-AH0,F1-ask-AE1,F1-at-AE1,F1-bags-AE1,F1-big-IH1,...,F1-toy-OY1,F1-train-EY1,F1-we-IY1,F1-wednesday-EH1,F1-wednesday-EY2,F1-wednesday-IY0,F1-will-AH0,F1-will-IH1,F1-with-IH0,F1-with-IH1
0,484.116707,0.000000,649.691041,479.885272,633.160121,567.639486,744.763339,539.505943,650.597091,418.221579,...,558.103092,598.918749,388.760823,659.521692,428.807428,0.000000,458.317544,0.0,451.351407,0.000000
1,446.479293,0.000000,805.901930,462.684377,594.366055,471.828542,767.568712,625.072530,854.488527,469.280507,...,526.556666,389.303757,429.566074,562.260227,0.000000,371.502363,376.116950,0.0,450.187307,0.000000
2,500.414386,0.000000,681.140780,457.321222,624.759445,0.000000,783.014323,688.876577,727.667387,551.872631,...,588.790482,494.794385,464.039105,628.624622,573.865729,0.000000,419.221923,0.0,0.000000,533.898484
3,454.650048,0.000000,597.563789,485.040418,0.000000,448.358869,721.693053,486.496701,621.065208,408.205085,...,486.987983,500.964263,363.266894,608.917180,516.772289,0.000000,455.938218,0.0,450.447378,0.000000
4,440.677259,0.000000,570.352779,367.708427,542.836571,0.000000,531.205255,439.593369,510.274918,396.962703,...,462.312784,551.478720,324.386679,550.618805,448.772615,0.000000,377.056530,0.0,439.580531,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,373.439207,436.228222,552.856185,413.165623,462.244063,427.028355,581.437571,554.111796,620.117427,409.366342,...,468.141905,400.395679,346.698229,442.615747,450.836784,0.000000,406.173443,0.0,404.690474,0.000000
636,382.289638,0.000000,504.845230,423.650958,609.150271,0.000000,628.395514,492.388026,584.881696,384.790654,...,438.897814,353.606437,319.910994,540.039985,440.448956,0.000000,344.062611,0.0,403.992729,0.000000
637,412.892373,0.000000,645.888058,482.797701,892.139839,1773.535156,676.025798,479.539749,658.410675,421.701828,...,629.698310,546.900984,303.396840,644.298475,406.628629,0.000000,698.005034,0.0,446.082325,0.000000
638,418.064048,0.000000,618.866782,286.899709,0.000000,553.390182,694.308547,363.337824,635.685699,408.574349,...,463.550769,366.878979,360.733320,401.566298,408.769797,0.000000,359.815566,0.0,430.033459,0.000000


In [8]:
def extract_country(location):
    l_list = location.split("_")
    if len(l_list) > 2:
        return l_list[2]
    elif len(l_list) > 1:
        return l_list[1]
    else:
        return l_list[0]
    
def extract_place(location):
    l_list = location.split("_")
    if len(l_list) > 1:
        return l_list[1]
    else:
        return l_list[0]

def word_find(feature):
    return feature.split("-")[0][0:2] + "-" + feature.split("-")[1]

dem_feats = ['age','gender']
in_df = input_data.input_df
in_df['location'] = in_df['location'].apply(extract_place)
in_df.gender = in_df.gender.replace(to_replace=['m', 'f'], value=[1, 0])
in_df['age'] = pd.to_numeric(in_df['age'])

opt_df = in_df.copy()[['id','gender','age','location']]

words = [word_find(f) for f in in_df.columns[4:]]
for w in words:
    for c in in_df.columns[4:]:
        if word_find(c) == w:
            opt_df[w] = in_df[c]
            break

in_list = list(in_df.columns[4:])
opt_list = list(opt_df.columns[4:])
for f in range(len(opt_list)):
    for s in range(len(opt_df)):
        if opt_df.iloc[s, f+4] == 0:
            for i in range(len(in_list)):
                word = word_find(in_list[i])
                if word == opt_list[f]:
                    if in_df.iloc[s, i+4] != 0.0:
                        opt_df.iloc[s, f+4] = in_df.iloc[s,i+4]
                        break
        else:
            continue
            
#normalize and filter for particular locations
new_df = opt_df.copy()[['id','gender','age','location']]
not_features = list(opt_df.columns)[0:4]

for c in opt_df.columns[4:]:
    if c[0:2] == "F1":
        name = "F1mF2" + c[2:]
        word = c.split("-")[1]
        c2 = "F2-" + word
        new_df[name] = opt_df[c2] - opt_df[c]
        
fs = list(new_df.columns)[4:]

for column in fs: 
#     new_df[column]=(new_df[column]-new_df[column].mean())/new_df[column].std()
    new_df[column]=(new_df[column]-new_df[column].min())/(new_df[column].max()-new_df[column].min())

opt_df = pd.merge(opt_df[not_features], new_df[fs], left_index=True, right_index=True)

places = ["uk", "texas", "oklahoma", "alabama", "mississippi", "georgia"]

filtered_df = opt_df.loc[opt_df['location'].isin(places)].reset_index(drop=True)

filtered_df

,id,gender,age,location,F1mF2-a,F1mF2-also,F1mF2-and,F1mF2-ask,F1mF2-at,F1mF2-bags,...,F1mF2-thick,F1mF2-things,F1mF2-three,F1mF2-to,F1mF2-toy,F1mF2-train,F1mF2-we,F1mF2-wednesday,F1mF2-will,F1mF2-with
0,english11,0,35,uk,0.441298,0.286420,0.394229,0.136644,0.444625,0.273916,...,0.546785,0.671384,0.424815,0.875343,0.342219,0.548640,0.283404,0.312891,0.330095,0.340731
1,english114,0,38,alabama,0.417357,0.179226,0.775684,0.317878,0.683404,0.422052,...,0.590509,0.758395,0.632733,0.559492,0.309356,0.782249,0.240518,0.412563,0.296950,0.296103
2,english116,1,56,georgia,0.486910,0.222956,0.614590,0.598257,0.522003,0.500622,...,0.646285,0.416178,0.608391,0.511590,0.110124,0.570956,0.338718,0.433908,0.269460,0.297132
3,english117,0,60,alabama,0.366811,0.006451,0.726806,0.665661,0.493186,0.702938,...,0.696635,0.682484,0.741579,0.564051,0.276153,0.642762,0.454729,0.434737,0.311165,0.430221
4,english12,0,71,uk,0.417679,0.353921,0.366947,0.367954,0.393462,0.388164,...,0.680430,0.711414,0.707590,0.637999,0.433091,0.585581,0.113859,0.099228,0.671141,0.454334
5,english135,1,22,alabama,0.217613,0.278152,0.513014,0.467692,0.519472,0.399120,...,0.578083,0.460348,0.572210,0.437337,0.515129,0.496650,0.270962,0.204285,1.000000,0.208882
6,english141,1,19,uk,0.372123,0.403375,0.344525,0.222353,0.528323,0.303895,...,0.492849,0.513048,0.444020,0.728123,0.376739,0.413222,0.385682,0.387963,0.325824,0.460733
7,english145,1,18,uk,0.293489,0.232952,0.437140,0.202228,0.332955,0.474640,...,0.612615,0.721040,0.640434,0.496336,0.361045,0.591180,0.325454,0.272377,0.323485,0.392535
8,english149,1,18,georgia,0.324891,0.383469,0.505314,0.454610,0.501631,0.583405,...,0.653621,0.685976,0.157116,0.457241,0.276866,0.307504,0.307868,0.418630,0.362233,0.306504
9,english157,1,35,texas,0.297747,0.171394,0.502315,0.586172,0.603783,0.600109,...,0.575280,0.475751,0.549228,0.528548,0.389369,0.554972,0.170777,0.170589,0.269367,0.289505


In [7]:
used_features = [f for f in list(filtered_df.columns[4:]) if f[0:5] in ["F1mF2"]]
filtered_features = dem_feats + used_features

filtered_df

,id,gender,age,location,F1mF2-a,F1mF2-also,F1mF2-and,F1mF2-ask,F1mF2-at,F1mF2-bags,...,F1mF2-thick,F1mF2-things,F1mF2-three,F1mF2-to,F1mF2-toy,F1mF2-train,F1mF2-we,F1mF2-wednesday,F1mF2-will,F1mF2-with
0,english11,0,35,uk,0.441298,0.286420,0.394229,0.136644,0.444625,0.273916,...,0.546785,0.671384,0.424815,0.875343,0.342219,0.548640,0.283404,0.312891,0.330095,0.340731
1,english114,0,38,alabama,0.417357,0.179226,0.775684,0.317878,0.683404,0.422052,...,0.590509,0.758395,0.632733,0.559492,0.309356,0.782249,0.240518,0.412563,0.296950,0.296103
2,english116,1,56,georgia,0.486910,0.222956,0.614590,0.598257,0.522003,0.500622,...,0.646285,0.416178,0.608391,0.511590,0.110124,0.570956,0.338718,0.433908,0.269460,0.297132
3,english117,0,60,alabama,0.366811,0.006451,0.726806,0.665661,0.493186,0.702938,...,0.696635,0.682484,0.741579,0.564051,0.276153,0.642762,0.454729,0.434737,0.311165,0.430221
4,english12,0,71,uk,0.417679,0.353921,0.366947,0.367954,0.393462,0.388164,...,0.680430,0.711414,0.707590,0.637999,0.433091,0.585581,0.113859,0.099228,0.671141,0.454334
5,english135,1,22,alabama,0.217613,0.278152,0.513014,0.467692,0.519472,0.399120,...,0.578083,0.460348,0.572210,0.437337,0.515129,0.496650,0.270962,0.204285,1.000000,0.208882
6,english141,1,19,uk,0.372123,0.403375,0.344525,0.222353,0.528323,0.303895,...,0.492849,0.513048,0.444020,0.728123,0.376739,0.413222,0.385682,0.387963,0.325824,0.460733
7,english145,1,18,uk,0.293489,0.232952,0.437140,0.202228,0.332955,0.474640,...,0.612615,0.721040,0.640434,0.496336,0.361045,0.591180,0.325454,0.272377,0.323485,0.392535
8,english149,1,18,georgia,0.324891,0.383469,0.505314,0.454610,0.501631,0.583405,...,0.653621,0.685976,0.157116,0.457241,0.276866,0.307504,0.307868,0.418630,0.362233,0.306504
9,english157,1,35,texas,0.297747,0.171394,0.502315,0.586172,0.603783,0.600109,...,0.575280,0.475751,0.549228,0.528548,0.389369,0.554972,0.170777,0.170589,0.269367,0.289505


In [14]:
m = h_model(data=filtered_df, features=filtered_features, y_feature="location", y_main="uk")
m.fit(NUM_SAMPLES=1)

Fitting Model of type rforest
Modeling Sample 1 of 1


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Accuracy: 0.942

Variable Importance Measurements:
         Feature  Importance
4      F1mF2-and    0.001331
56    F1mF2-with    0.000814
3     F1mF2-also    0.000740
46   F1mF2-these    0.000296
5      F1mF2-ask    0.000074
23    F1mF2-into    0.000074
15  F1mF2-cheese    0.000000
1         gender    0.000000
0            age    0.000000
18   F1mF2-fresh    0.000000


In [ ]:
m2 = h_model(data=filtered_df, features=filtered_features, y_feature="location", y_main="texas")
m2.fit(model_type="ridge_classification", NUM_SAMPLES=10)